In [11]:
!pip install gradio

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv(r'/content/weather.csv')

# Prepare target variable
temperature = data['RainTomorrow'].values  # Extract 'RainTomorrow' values
data['RainTomorrow'] = data['RainTomorrow'].map({'No': 0, 'Yes': 1})  # Map to numerical
temperature = data['RainTomorrow'].values  # Update temperature with mapped values

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaled = scaler.fit_transform(temperature.reshape(-1, 1)) # Uncommented and fixed

# Create sequences
sequence_length = 30
X, y = [], []
for i in range(sequence_length, len(temperature_scaled)):
    X.append(temperature_scaled[i - sequence_length:i, 0])
    y.append(temperature_scaled[i, 0])

X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train model
model = Sequential([
    SimpleRNN(50, activation='tanh', input_shape=(X_train.shape[1], 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=2)

# Predict and inverse transform
predicted_temperatures = model.predict(X_test)
predicted_temperatures = scaler.inverse_transform(predicted_temperatures) # Reshape done by inverse_transform
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 - 2s - 244ms/step - loss: 0.1938
Epoch 2/20
9/9 - 0s - 21ms/step - loss: 0.1599
Epoch 3/20
9/9 - 0s - 8ms/step - loss: 0.1565
Epoch 4/20
9/9 - 0s - 8ms/step - loss: 0.1459
Epoch 5/20
9/9 - 0s - 15ms/step - loss: 0.1443
Epoch 6/20
9/9 - 0s - 8ms/step - loss: 0.1442
Epoch 7/20
9/9 - 0s - 8ms/step - loss: 0.1432
Epoch 8/20
9/9 - 0s - 8ms/step - loss: 0.1403
Epoch 9/20
9/9 - 0s - 15ms/step - loss: 0.1415
Epoch 10/20
9/9 - 0s - 8ms/step - loss: 0.1394
Epoch 11/20
9/9 - 0s - 8ms/step - loss: 0.1376
Epoch 12/20
9/9 - 0s - 8ms/step - loss: 0.1387
Epoch 13/20
9/9 - 0s - 8ms/step - loss: 0.1374
Epoch 14/20
9/9 - 0s - 8ms/step - loss: 0.1387
Epoch 15/20
9/9 - 0s - 15ms/step - loss: 0.1365
Epoch 16/20
9/9 - 0s - 11ms/step - loss: 0.1336
Epoch 17/20
9/9 - 0s - 9ms/step - loss: 0.1345
Epoch 18/20
9/9 - 0s - 9ms/step - loss: 0.1331
Epoch 19/20
9/9 - 0s - 15ms/step - loss: 0.1313
Epoch 20/20
9/9 - 0s - 9ms/step - loss: 0.1280
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
import gradio as gr

# ... (your existing code for loading data, preprocessing, and model training) ...

# Function to make predictions (modified for Gradio)
def predict_weather_gradio(input_data):
    """Predicts the weather based on the input data.

    Args:
        input_data (list): A list of 30 past weather values (0 or 1).

    Returns:
        float: The predicted weather for the next day (0 or 1).
    """
    input_data = np.array(input_data).reshape(1, sequence_length, 1)
    predicted_scaled = model.predict(input_data)
    predicted_weather = scaler.inverse_transform(predicted_scaled)[0][0]
    # Round to 0 or 1
    predicted_weather = round(predicted_weather)
    return predicted_weather

# Create Gradio interface
# Updated to use gr.DataFrame instead of gr.inputs.Dataframe
iface = gr.Interface(
    fn=predict_weather_gradio,
    inputs=gr.DataFrame(
        headers=["Day " + str(i) for i in range(1, sequence_length + 1)],
        datatype="number",
        row_count=1,
        col_count=sequence_length,
    ),
    outputs="number",  # Output is a single number (0 or 1)
    title="Weather Prediction",
    description="Enter the past 30 days of 'RainTomorrow' values (0 for No, 1 for Yes) to predict the next day's weather.",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c8bd987298dc5ceae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
